# 👥 Module 8: Multi-Agent Patterns

**AI Agent Architectures Workshop - Day 2**

This notebook covers:
- Sequential Pattern (Employee Onboarding Crew)
- Supervisor Pattern (Sales Analyst + Researcher)
- Reflection/Round Robin Pattern (Writer + Critic)
- Decision Calculus for Multi-Agent Design

**Prerequisites:** Run `00_setup.ipynb` first.

In [ ]:
!pip install openai python-dotenv --quiet

In [ ]:
import os
import json

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: https://xxx.openai.azure.com/
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

## 1. Sequential Pattern: Employee Onboarding Crew

Agents execute in sequence, each passing output to the next.

In [ ]:
class SequentialAgent:
    def __init__(self, name: str, role: str, instructions: str):
        self.name = name
        self.role = role
        self.instructions = instructions
    
    def execute(self, input_data: str) -> str:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": f"You are {self.name}, a {self.role}. {self.instructions}"},
                {"role": "user", "content": input_data}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content

# Define onboarding crew
hr_agent = SequentialAgent(
    name="HR Coordinator",
    role="HR specialist at a bank",
    instructions="Create an onboarding checklist for the new employee. Include compliance training, system access, and team introductions."
)

it_agent = SequentialAgent(
    name="IT Setup Specialist",
    role="IT administrator",
    instructions="Based on the HR checklist, specify the exact systems, permissions, and security training needed. Include banking-specific systems."
)

compliance_agent = SequentialAgent(
    name="Compliance Officer",
    role="banking compliance specialist",
    instructions="Review the onboarding plan and add required regulatory training (AML, KYC, data privacy). Flag any compliance gaps."
)

print("✅ Sequential agents defined")

In [ ]:
def run_sequential_crew(new_employee: dict) -> dict:
    """Run sequential onboarding crew"""
    results = {"employee": new_employee, "stages": []}
    
    # Stage 1: HR creates initial checklist
    print("🧑‍💼 HR Coordinator creating checklist...")
    hr_input = f"New employee: {new_employee['name']}, Role: {new_employee['role']}, Department: {new_employee['department']}"
    hr_output = hr_agent.execute(hr_input)
    results["stages"].append({"agent": "HR", "output": hr_output})
    print(f"   ✓ HR checklist complete")
    
    # Stage 2: IT adds technical requirements
    print("💻 IT Specialist adding system requirements...")
    it_input = f"HR Checklist:\n{hr_output}\n\nEmployee role: {new_employee['role']}"
    it_output = it_agent.execute(it_input)
    results["stages"].append({"agent": "IT", "output": it_output})
    print(f"   ✓ IT requirements complete")
    
    # Stage 3: Compliance review
    print("📋 Compliance Officer reviewing...")
    compliance_input = f"Onboarding Plan:\n{hr_output}\n\nIT Requirements:\n{it_output}"
    compliance_output = compliance_agent.execute(compliance_input)
    results["stages"].append({"agent": "Compliance", "output": compliance_output})
    print(f"   ✓ Compliance review complete")
    
    return results

# Test with new employee
new_hire = {
    "name": "Sarah Johnson",
    "role": "Senior Loan Officer",
    "department": "Commercial Lending"
}

results = run_sequential_crew(new_hire)
print("\n" + "="*50)
print("FINAL COMPLIANCE REVIEW:")
print("="*50)
print(results["stages"][-1]["output"])

## 2. Supervisor Pattern: Sales Analyst + Market Researcher

A supervisor agent coordinates specialist agents and synthesizes their outputs.

In [ ]:
class SupervisorCrew:
    def __init__(self):
        self.supervisor_prompt = """You are a Sales Team Supervisor at a bank. 
You coordinate a team of specialists:
- Sales Analyst: Analyzes customer data and sales performance
- Market Researcher: Researches market trends and competitor offerings

Your job is to:
1. Delegate tasks to the right specialist
2. Synthesize their findings into actionable recommendations
3. Present a unified strategy to leadership"""
        
        self.sales_analyst_prompt = """You are a Sales Analyst at a bank.
Analyze customer segments, sales metrics, and identify opportunities.
Focus on data-driven insights about customer behavior and product performance."""
        
        self.market_researcher_prompt = """You are a Market Researcher at a bank.
Research market trends, competitor products, and industry developments.
Focus on external factors affecting the banking market."""
    
    def _call_agent(self, system_prompt: str, user_message: str) -> str:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content
    
    def analyze(self, business_question: str) -> dict:
        results = {"question": business_question}
        
        # Supervisor delegates to specialists
        print("👔 Supervisor delegating tasks...")
        
        # Sales Analyst
        print("📊 Sales Analyst working...")
        sales_analysis = self._call_agent(
            self.sales_analyst_prompt,
            f"Analyze this business question from a sales perspective: {business_question}"
        )
        results["sales_analysis"] = sales_analysis
        
        # Market Researcher
        print("🔍 Market Researcher working...")
        market_research = self._call_agent(
            self.market_researcher_prompt,
            f"Research market context for: {business_question}"
        )
        results["market_research"] = market_research
        
        # Supervisor synthesizes
        print("👔 Supervisor synthesizing findings...")
        synthesis_prompt = f"""Based on your team's findings, provide a strategic recommendation.

SALES ANALYST FINDINGS:
{sales_analysis}

MARKET RESEARCHER FINDINGS:
{market_research}

Synthesize these into 3-5 actionable recommendations for leadership."""
        
        final_recommendation = self._call_agent(self.supervisor_prompt, synthesis_prompt)
        results["recommendation"] = final_recommendation
        
        return results

# Test supervisor pattern
crew = SupervisorCrew()
question = "How should we expand our mortgage product offerings to millennials?"

results = crew.analyze(question)
print("\n" + "="*50)
print("SUPERVISOR RECOMMENDATION:")
print("="*50)
print(results["recommendation"])

## 3. Reflection Pattern: Writer + Critic (Round Robin)

Agents iteratively improve output through feedback loops.

In [ ]:
class ReflectionCrew:
    def __init__(self, max_iterations: int = 3):
        self.max_iterations = max_iterations
        
        self.writer_prompt = """You are a Marketing Copywriter at a bank.
Write compelling, compliant marketing copy for banking products.
Focus on benefits, clarity, and regulatory compliance."""
        
        self.critic_prompt = """You are a Marketing Compliance Reviewer at a bank.
Review marketing copy for:
1. Regulatory compliance (no misleading claims)
2. Clarity and readability
3. Effectiveness and persuasiveness
4. Required disclosures

Provide specific, actionable feedback. If the copy is acceptable, respond with 'APPROVED'."""
    
    def _call_agent(self, system_prompt: str, user_message: str) -> str:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content
    
    def create_copy(self, product_brief: str) -> dict:
        results = {"brief": product_brief, "iterations": []}
        
        # Initial draft
        print("✍️ Writer creating initial draft...")
        current_copy = self._call_agent(
            self.writer_prompt,
            f"Write marketing copy for: {product_brief}"
        )
        
        for i in range(self.max_iterations):
            print(f"\n🔄 Iteration {i+1}/{self.max_iterations}")
            
            # Critic reviews
            print("   🔍 Critic reviewing...")
            feedback = self._call_agent(
                self.critic_prompt,
                f"Review this marketing copy:\n\n{current_copy}"
            )
            
            results["iterations"].append({
                "iteration": i + 1,
                "copy": current_copy,
                "feedback": feedback
            })
            
            # Check if approved
            if "APPROVED" in feedback.upper():
                print("   ✅ Copy approved!")
                results["final_copy"] = current_copy
                results["status"] = "approved"
                return results
            
            # Writer revises based on feedback
            print("   ✍️ Writer revising...")
            revision_prompt = f"""Revise this marketing copy based on the feedback:

CURRENT COPY:
{current_copy}

FEEDBACK:
{feedback}

Create an improved version addressing all feedback points."""
            
            current_copy = self._call_agent(self.writer_prompt, revision_prompt)
        
        results["final_copy"] = current_copy
        results["status"] = "max_iterations_reached"
        return results

# Test reflection pattern
crew = ReflectionCrew(max_iterations=3)
brief = "High-yield savings account with 4.5% APY, no minimum balance, FDIC insured"

results = crew.create_copy(brief)
print("\n" + "="*50)
print(f"FINAL COPY (Status: {results['status']}):")
print("="*50)
print(results["final_copy"])

## 4. Decision Calculus: When to Use Multi-Agent

Use this framework to decide if multi-agent is appropriate.

In [ ]:
def multi_agent_decision(task_description: str) -> dict:
    """Analyze whether a task needs multi-agent architecture"""
    
    analysis_prompt = """Analyze this task and recommend whether to use single-agent or multi-agent architecture.

Consider:
1. Task complexity (simple vs requires multiple expertise areas)
2. Parallelization potential (can subtasks run simultaneously?)
3. Quality requirements (does it need review/iteration?)
4. Cost sensitivity (multi-agent = more tokens)

Respond in JSON format:
{
    "recommendation": "single-agent" or "multi-agent",
    "pattern": "sequential" or "supervisor" or "reflection" or "none",
    "reasoning": "explanation",
    "estimated_agents": number,
    "cost_multiplier": number
}"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": analysis_prompt},
            {"role": "user", "content": f"Task: {task_description}"}
        ],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)

# Test with different banking tasks
tasks = [
    "Answer a customer question about their account balance",
    "Create a comprehensive loan application review with credit, income, and compliance checks",
    "Write and review marketing copy for a new credit card product",
    "Investigate a complex fraud case involving multiple accounts"
]

print("=== Multi-Agent Decision Analysis ===")
for task in tasks:
    result = multi_agent_decision(task)
    print(f"\n📋 Task: {task[:50]}...")
    print(f"   Recommendation: {result['recommendation']}")
    print(f"   Pattern: {result['pattern']}")
    print(f"   Agents: {result['estimated_agents']}, Cost: {result['cost_multiplier']}x")

## Summary

**Multi-Agent Patterns:**

| Pattern | Use Case | Banking Example |
|---------|----------|----------------|
| Sequential | Pipeline processing | Employee onboarding |
| Supervisor | Coordinated specialists | Sales analysis team |
| Reflection | Quality improvement | Marketing copy review |

**Decision Calculus:**
- Use single-agent for simple, focused tasks
- Use multi-agent when task requires multiple expertise areas
- Consider cost: multi-agent = 2-5x token usage